# ImperativeDeclarative

Most situations in computer science involve imperative programming. Consider a simple Python program (Example 2-20).

Example 2-20. Python program imperatively performing an addition

In [ ]:
a = 3
b = 4
c = a + b
c

Output:<br>`7`

This program, when translated into machine code, instructs the machine to perform a primitive addition operation on two registers, one containing 3, and the other containing 4. The result is then 7. This style of programming is called imperative since the program tells the computer explicitly which actions to perform.

An alternative style of programming is declarative. In a declarative system, a computer program is a high-level description of the computation that is to be performed. It does not instruct the computer exactly how to perform the computation. Example 2-21 is the TensorFlow equivalent of Example 2-20.

Example 2-21. TensorFlow program declaratively performing an addition

In [ ]:
a = tf.constant(3)
b = tf.constant(4)
c = a + b
c

Output:<br>`<tf.Tensor 'add_1:0' shape=() dtype=int32>`

In [ ]:
c.eval()

Output:<br>`7`

Note that the value of c isn’t 7! Rather, it’s a symbolic tensor. This code specifies the computation of adding two values together to create a new tensor. The actual computation isn’t executed until we call c.eval(). In the sections before, we have been using the eval() method to simulate imperative style in TensorFlow since it can be challenging to understand declarative programming at first.

However, declarative programming is by no means an unknown concept to software engineering. Relational databases and SQL provide an example of a widely used declarative programming system. Commands like SELECT and JOIN may be implemented in an arbitrary fashion under the hood so long as their basic semantics are preserved. TensorFlow code is best thought of as analogous to a SQL program; the TensorFlow code specifies a computation to be performed, with details left up to TensorFlow. The TensorFlow developers exploit this lack of detail under the hood to tailor the execution style to the underlying hardware, be it CPU, GPU, or mobile device.

It’s important to note that the grand weakness of declarative programming is that the abstraction is quite leaky. For example, without detailed understanding of the underlying implementation of the relational database, long SQL programs can become unbearably inefficient. Similarly, large TensorFlow programs implemented without understanding of the underlying learning algorithms are unlikely to work well. In the rest of this section, we will start paring back the abstraction, a process we will continue throughout the rest of the book.

<div style="border: 1px solid black; padding: 10px;"><b style="font-size: 2em;">TensorFlow Eager</b><br> The TensorFlow team recently added a new experimental module, TensorFlow Eager, that enables users to run TensorFlow calculations imperatively. In time, this module will likely become the preferred entry mode for new programmers learning TensorFlow. However, at the timing of writing, this module is still very new with many rough edges. As a result, we won’t teach you about Eager mode, but encourage you to check it out for yourself.<br>It’s important to emphasize that much of TensorFlow will remain declarative even after Eager matures, so it’s worth learning declarative TensorFlow regardless.</div>

# TensorFlow Graphs

Any computation in TensorFlow is represented as an instance of a tf.Graph object. Such a graph consists of a set of instances of tf.Tensor objects and tf.Operation objects. We have covered tf.Tensor in some detail, but what are tf.Operation objects? You have already seen them over the course of this chapter. A call to an operation like tf.matmul creates a tf.Operation instance to mark the need to perform the matrix multiplication operation.

When a tf.Graph is not explicitly specified, TensorFlow adds tensors and operations to a hidden global tf.Graph instance. This instance can be fetched by tf.get_default_graph() (Example 2-22).

Example 2-22. Getting the default TensorFlow graph

In [ ]:
tf.get_default_graph()

Output:<br>`<tensorflow.python.framework.ops.Graph>`

It is possible to specify that TensorFlow operations should be performed in graphs other than the default. We will demonstrate examples of this in future chapters.

# TensorFlow Sessions

In TensorFlow, a tf.Session() object stores the context under which a computation is performed. At the beginning of this chapter, we used tf.InteractiveSession() to set up an environment for all TensorFlow computations. This call created a hidden global context for all computations performed. We then used tf.Tensor.eval() to execute our declaratively specified computations. Underneath the hood, this call is evaluated in context of this hidden global tf.Session. It can be convenient (and often necessary) to use an explicit context for a computation instead of a hidden context (Example 2-23).

Example 2-23. Explicitly manipulating TensorFlow sessions

In [ ]:
sess = tf.Session()
a = tf.ones((2, 2))
b = tf.matmul(a, a)
b.eval(session=sess)

Output:<br>`
array([[ 2., 2.],
       [ 2., 2.]], dtype=float32)
    `

This code evaluates b in the context of sess instead of the hidden global session. In fact, we can make this more explicit with an alternate notation (Example 2-24).

Example 2-24. Running a computation within a session

In [ ]:
sess.run(b)

Output:<br>`
array([[ 2., 2.],
       [ 2., 2.]], dtype=float32)
    `

In fact, calling b.eval(session=sess) is just syntactic sugar for calling sess.run(b).

This entire discussion may smack a bit of sophistry. What does it matter which session is in play given that all the different methods seem to return the same answer? Explicit sessions don’t really show their value until you start to perform computations that have state, a topic you will learn about in the following section.

# TensorFlow Variables

All the example code in this section has used constant tensors. While we could combine and recombine these tensors in any way we chose, we could never change the value of tensors themselves (only create new tensors with new values). The style of programming so far has been functional and not stateful. While functional computations are very useful, machine learning often depends heavily on stateful computations. Learning algorithms are essentially rules for updating stored tensors to explain provided data. If it’s not possible to update these stored tensors, it would be hard to learn.

The tf.Variable() class provides a wrapper around tensors that allows for stateful computations. The variable objects serve as holders for tensors. Creating a variable is easy enough (Example 2-25).

Example 2-25. Creating a TensorFlow variable

In [ ]:
a = tf.Variable(tf.ones((2, 2)))
a

Output:<br>`<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32_ref>`

What happens when we try to evaluate the variable a as though it were a tensor, as in Example 2-26?

Example 2-26. Evaluating an uninitialized variable fails

In [ ]:
a.eval()

Output:<br>`FailedPreconditionError: Attempting to use uninitialized value Variable`

The evaluation fails since variables have to be explicitly initialized. The easiest way to initialize all variables is to invoke tf.global_variables_initializer. Running this operation within a session will initialize all variables in the program (Example 2-27).

Example 2-27. Evaluating initialized variables

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
a.eval(session=sess)

Output:<br>`
array([[ 1., 1.],
       [ 1., 1.]], dtype=float32)
    `

After initialization, we are able to fetch the value stored within the variable as though it were a plain tensor. So far, there’s not much more to variables than plain tensors. Variables only become interesting once we can assign to them. tf.assign() lets us do this. Using tf.assign() we can update the value of an existing variable (Example 2-28).

Example 2-28. Assigning values to variables

In [ ]:
sess.run(a.assign(tf.zeros((2,2))))

Output:<br>`
array([[ 0., 0.],
       [ 0., 0.]], dtype=float32)
        `

In [ ]:
sess.run(a)

Output:<br>`
array([[ 0., 0.],
       [ 0., 0.]], dtype=float32)
    `

What would happen if we tried to assign a value to variable a not of shape (2,2)? Let’s find out in Example 2-29.

Example 2-29. Assignment fails when shapes aren’t equal

In [ ]:
sess.run(a.assign(tf.zeros((3,3))))

Output:<br>`
ValueError: Dimension 0 in both shapes must be equal, but are 2 and 3 for 'Assign_3'
(op: 'Assign') with input shapes: [2,2], [3,3].
    `

You can see that TensorFlow complains. The shape of the variable is fixed upon initialization and must be preserved with updates. As another interesting note, tf.assign is itself a part of the underlying global tf.Graph instance. This allows TensorFlow programs to update their internal state every time they are run. We will make heavy use of this feature in the chapters to come.